In [ ]:
class Strategy:
    def __init__(self, data) -> None:
        self.data = data
        self.indicator = None

data = [0,1,2,3,4,5]
strat = Strategy(data)
strat.__dict__.items()

In [ ]:
print(len.__name__)
print(len.__class__.__name__)

In [ ]:
import datetime as dt 
import yfinance as yf
import pandas as pd

num_days = 730
start=dt.date(2021, 1, 1)
start = max(start, dt.date.today() - dt.timedelta(days=num_days-1))
end = dt.date.today() 
print(start, end)
data = yf.download("SPY", interval="1h", start=start, end=end, prepost=True)
data = data.drop(['Adj Close'], axis=1)
data.index = list(map(lambda x: dt.datetime.strptime(str(x).replace(":",""), '%Y-%m-%d %H%M%S%z').replace(tzinfo=None), data.index))
data = data.rename_axis("Datetime").reset_index()
print(data)


In [ ]:
timestr = '2022-07-28 04:00:00'
output = dt.datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S')
print(output)


In [ ]:
import bokeh.colors

# for c in dir(bokeh.colors.named):
    # print(c, c.__class__)
import bokeh.palettes
print(bokeh.palettes.Dark2_8)

# print(bokeh.colors.named.blue.__class__)
# print(all_colors)

# Top gainers aftermath

In [ ]:
from tradingview_screener import *
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

n_rows, df = (Query()
 .select('name', 'close', 'change', 'volume', 'relative_volume_10d_calc', 'market_cap_basic', 'float_shares_outstanding')
 .where(
     Column('market_cap_basic') >= 1_000_000,
     Column('relative_volume_10d_calc') > 1.2,
     Column('change') > 5,
     Column('volume') > 1E6,
     Column('close') > 2,
    )
 .order_by('change', ascending=False)
 .get_scanner_data())

df.volume = df.volume * 1E-6
df.market_cap_basic = df.market_cap_basic * 1E-6
df.float_shares_outstanding = df.float_shares_outstanding * 1E-6
df = df.rename(columns={
    "volume": "Volume (M)", 
    "market_cap_basic": "Market Cap (M)",
    "change": "Change %",
    "float_shares_outstanding": "Float (M)"
    })
df = df[df.ticker.str.contains("NASDAQ:") | df.ticker.str.contains("NYSE:")]
print(len(df))
display(df)


In [ ]:
import datetime as dt
from pandas.tseries.offsets import BDay

(dt.date(2024, 9, 1) - BDay(0)).date()

In [ ]:
import datetime as dt
from pandas.tseries.offsets import BDay

# all_data = {dt.date.today() : df.ticker.to_list()}
fake_date_range = [dt.date(2024, 9, i) for i in range(1, 30)]
fake_data = {fake_date_range[i//2] : [str(j) for j in range(i, i+3)] for i in range(0, len(fake_date_range)*2,2)}
print(fake_data)


observation_time = 1 # days after making top gainer list
for today_date in fake_date_range:
    stocks_to_scan = set()
    for time_delta in range(observation_time + 1):
        # lookback_date = (today_date - BDay(time_delta)).date()
        lookback_date = today_date - dt.timedelta(days=time_delta)
        print(lookback_date)
        if lookback_date in fake_data:
            stocks_to_scan.update(fake_data[lookback_date])
            print("updated:", fake_data[lookback_date])
    print(today_date, stocks_to_scan)
    print()
        

# Premarket

In [ ]:
from tradingview_screener import *
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

n_rows, df = (
    Scanner.premarket_gainers
    .select('name', 'close', 'volume', 'premarket_change', 'premarket_volume', 'relative_volume_10d_calc', 'market_cap_basic', 'float_shares_outstanding')
    .where(
     Column('market_cap_basic') >= 1_000_000,
     Column('relative_volume_10d_calc') > 1.2,
     Column('premarket_change') > 5,
     Column('premarket_volume') > 1E6,
     Column('close') > 1
    )
    .get_scanner_data())


df.volume = df.volume * 1E-6
df.market_cap_basic = df.market_cap_basic * 1E-6
df.premarket_volume = df.premarket_volume * 1E-6
df.float_shares_outstanding = df.float_shares_outstanding * 1E-6
df = df.rename(columns={
    "volume": "Volume (M)", 
    "market_cap_basic": "Market Cap (M)",
    "premarket_change": "Premarket %",
    "premarket_change_abs": "Premarket $",
    "premarket_volume": "Premarket Volume (M)",
    "float_shares_outstanding": "Float (M)"
    })
print(n_rows)

display(df)

In [ ]:
import datetime as dt
from download import *
from visualize import *

yf_df = download_from_yf("EBS", "1m", dt.date(2024,8,28), dt.date(2024,8,30))


In [ ]:
# pd.set_option('display.max_rows', 2500)
display(yf_df)

In [ ]:
import datetime as dt
from download import *
from visualize import *

polygon_df = download_from_polygon("EBS", "1m", dt.date(2024,8,28), dt.date(2024,8,30))


In [ ]:
display(polygon_df)

In [ ]:
pd.set_option('display.max_rows', 3000)
merged_df = yf_df.merge(polygon_df, how='left')
display(merged_df)
pd.set_option('display.max_rows', 10)

In [ ]:
market_open = pd.to_datetime('09:30:00').time()
market_close = pd.to_datetime('16:00:00').time()

pre_post_data = polygon_df.loc[(polygon_df['Datetime'].dt.time <= market_open) | (polygon_df['Datetime'].dt.time >= market_close)]

# TODO: Lookup not by exact value, but by the closest timestamp in yf_df after the query timestamp. 
for index, row in pre_post_data.iterrows():
    yf_row = yf_df[yf_df['Datetime'] == row.Datetime]
    assert len(yf_row) <= 1, "Multiple of the same timestamps found."
    yf_df.loc[yf_row.index, 'Volume'] = row.Volume
    

# pre_post_data["closest_timestamp"] = pd.merge_asof(pre_post_data, yf_df, on='Datetime', direction='forward')['Datetime']

In [ ]:
import pandas_market_calendars as mcal
import pandas as pd
import datetime as dt

market_schedule = mcal.get_calendar('NYSE').schedule(start_date='2024-01-01', end_date='2024-12-31')
regular_business_days = market_schedule.loc[market_schedule['market_close'].dt.time >= pd.to_datetime('20:00:00').time()].index.to_list()

regular_business_days = [i.to_pydatetime().date() for i in regular_business_days]
print(regular_business_days)
print((dt.datetime.now().date() + dt.timedelta(days=1)) in regular_business_days)

In [ ]:
import yfinance as yf
import datetime as dt
import requests
import pandas as pd

interval = 'Daily' # '5min'
# url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval={interval}&outputsize=full&apikey=demo"
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&outputsize=full&apikey=demo"
r = requests.get(url)
data = r.json()

# data = yf.download('IBM', interval='1d', start=dt.datetime(1914,1,1), end=dt.datetime.today(), prepost=True, progress=False)
# display(data)

df = pd.DataFrame(columns=["Datetime", 'Open', 'High', 'Low', 'Close', 'Volume'])
for timestamp, ohlcv in data[f"Time Series ({interval})"].items():
    df = pd.concat([pd.DataFrame([[dt.datetime.fromisoformat(timestamp),
                     ohlcv['1. open'],
                     ohlcv['2. high'],
                     ohlcv['3. low'],
                     ohlcv['4. close'],
                     ohlcv['5. volume']]], columns=df.columns), df], ignore_index=True)
display(df)

In [ ]:
from download import * 

df = get_historical_data("EBS", "1m", save_to_pickle=False)

In [ ]:
pd.set_option('display.max_rows', 2500)
display(df)
pd.set_option('display.max_rows', 100)

In [ ]:
from bisect import bisect_left
import datetime as dt
from download import * 
for query_date in [dt.date(2024, 1, 1) + dt.timedelta(days=i) for i in range(365)]:
    result = most_recent_business_day(query_date)
    print(query_date, result)

In [ ]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

file = 'C:\\Users\\georg\\GitHub\\money-tree\\test_run.html'

gauth = GoogleAuth(settings_file='C:\\Users\\georg\\GitHub\\money-tree\\settings.yaml', http_timeout=30)
drive = GoogleDrive(gauth)
upload_file_list = [file] 
for upload_file in upload_file_list: 
    gfile = drive.CreateFile({'parents': [{'id': '1N_7YUWKB2ezW_d_fse719Xo_toFuqx0q'}]}) # Read file and set it as the content of this instance. 
    gfile.SetContentFile(upload_file) 
    gfile.Upload() # Upload the file.

# list file
file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format('1N_7YUWKB2ezW_d_fse719Xo_toFuqx0q')}).GetList() 
for file in file_list: 
    print('title: %s, id: %s' % (file['title'], file['id']))

# # download file
# for i, file in enumerate(sorted(file_list, key = lambda x: x['title']), start=1):
#     print('Downloading {} file from GDrive ({}/{})'.format(file['title'], i, len(file_list))) 
#     file.GetContentFile(file['title'])